# Selectors

Selectors are powerful tools to filter atoms and create subsets of your data. They work like database queries for your molecules.

**Why use Selectors?**
- **Filtering**: Select atoms by element, type, or index.
- **Geometry**: Select atoms within a region or distance.
- **Composition**: Combine criteria with AND (`&`), OR (`|`), NOT (`~`).

---


## 1. Basic Selectors

Select by intrinsic properties like element or type.


In [ ]:
import molpy as mp
from molpy.core.selector import ElementSelector, AtomTypeSelector, AtomIndexSelector

# Create a dummy frame
frame = mp.Frame()
frame["atoms"] = mp.Block(
    {
        "element": ["C", "C", "H", "H", "O"],
        "type": [1, 1, 2, 2, 3],
        "x": [0.0, 1.0, 2.0, 3.0, 4.0],
        "y": [0.0, 0.0, 0.0, 0.0, 0.0],
        "z": [0.0, 0.0, 0.0, 0.0, 0.0],
    }
)
atoms = frame["atoms"]

# Select Carbons
sel_c = ElementSelector("C")
carbons = sel_c(atoms)
print(f"Carbons: {carbons.nrows}")

# Select Type 2 (Hydrogens)
sel_h = AtomTypeSelector(2)
hydrogens = sel_h(atoms)
print(f"Type 2: {hydrogens.nrows}")

## 2. Geometric Selectors

Select based on spatial coordinates.


In [ ]:
from molpy.core.selector import CoordinateRangeSelector, DistanceSelector

# Select atoms with x > 1.5
sel_x = CoordinateRangeSelector(axis="x", min_value=1.5)
right_side = sel_x(atoms)
print(f"Atoms with x > 1.5: {right_side['element']}")

# Select atoms within 1.5 Å of origin
sel_dist = DistanceSelector(center=[0.0, 0.0, 0.0], max_distance=1.5)
close_atoms = sel_dist(atoms)
print(f"Close atoms: {close_atoms['element']}")

## 3. Combining Selectors

Combine logic using standard operators: `&` (AND), `|` (OR), `~` (NOT).


In [ ]:
# (Carbon OR Oxygen) AND (x > 0.5)
complex_sel = (ElementSelector("C") | ElementSelector("O")) & CoordinateRangeSelector(
    "x", min_value=0.5
)

result = complex_sel(atoms)
print(f"Complex selection: {result['element']} at x={result['x']}")

# NOT Hydrogen
no_h = ~ElementSelector("H")
print(f"Non-hydrogens: {no_h(atoms)['element']}")

## 4. Using Masks

Selectors can return a boolean mask instead of a new Block.


In [ ]:
mask = sel_c.mask(atoms)
print(f"Boolean mask: {mask}")

# Use mask with numpy/pandas logic if needed
import numpy as np

print(f"Indices: {np.where(mask)[0]}")